In [6]:
%load_ext autoreload
%autoreload 2


In [ ]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [ ]:
from coco import ImageCaptioningModel, CocoDataModule, collate_fn
from adapter_utils import FTConf


In [ ]:
data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=32,
)
model = ImageCaptioningModel(
    "google/vit-base-patch16-224-in21k",
    "bert-base-uncased",
    data_module=data_module,
    learning_rate=1e-4,
    peft_conf=FTConf(ft_type="full")
)


In [ ]:
model = model.load_from_checkpoint(
    "/data/home/ngjhn/read/lightning_logs/epoch=1-val_loss=0.80-other_metric=0.00.ckpt", 
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    text_model_name_or_path="bert-base-uncased", 
    data_module=data_module, peft_conf=FTConf(ft_type="full"))


In [ ]:
data_module.setup("test")

all_results = []
for batch in data_module.test_dataloader():
    generated_ids = model.model.generate(batch["pixel_values"])
    generated_text = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    results = [{'image_id': image_id, 'caption': caption} for image_id, caption in zip(batch["image_id"], generated_text)]
    all_results.extend(results)


In [ ]:
model._evaluate(results)